In [1]:
%load_ext autoreload
%autoreload 2

from volume_fish import VolumeFish
from caiman_process import caiman_mcorr
from mesm import *
from utils import visualize_images, visualize_volumes
from tifffile import memmap
import pandas as pd

pd.options.display.max_colwidth = 120

### Change the exp_folder to the path of the experiment

In [2]:
exp_folder = r'C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024'

In [3]:
#Create fish

fish = VolumeFish(exp_folder)
fish.data_paths

Batch exists. Loading batch.pickle


{'injection': WindowsPath('C:/Users/minel/Documents/data/vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024/injection.txt'),
 'log': WindowsPath('C:/Users/minel/Documents/data/vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024/log.123.txt'),
 'postgavage_path': WindowsPath('C:/Users/minel/Documents/data/vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024/postgavage_vagal_R'),
 'cropped': WindowsPath('C:/Users/minel/Documents/data/vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024/postgavage_vagal_R/img_cropped.tif'),
 'mesmerize': WindowsPath('C:/Users/minel/Documents/data/vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024/postgavage_vagal_R/mesmerize-batch'),
 'opts': WindowsPath('C:/Users/minel/Documents/data/vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024/postgavage_vagal_R/opts.pkl'),
 'raw_image': WindowsPath('C:/Users/minel/Documents/data/vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_2

## Pick a plane to apply movement correction

In [6]:
iw = visualize_volumes(fish)
iw.show()

RFBOutputContext()

In [7]:
iw.plot.canvas.close()

In [12]:
plane = 3

## Set parameters

In [13]:
# motion correction parameters
strides = (40, 40)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (15, 15)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (20, 20)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 20     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

In [14]:
opts_dict = {'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid}

## Motion correct

In [15]:
mcorr_img = caiman_mcorr(fish, plane=plane, **opts_dict)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_8dpf_fed_glucose_625mM_54_20230405\postgavage_vagal_R\mesmerize-batch\f99dc1da-ec92-4301-8465-873eb33f8f28\image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.26it/s]


Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_8dpf_fed_glucose_625mM_54_20230405\postgavage_vagal_R\mesmerize-batch\f99dc1da-ec92-4301-8465-873eb33f8f28\image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_8dpf_fed_glucose_625mM_54_20230405\postgavage_vagal_R\mesmerize-batch\f99dc1da-ec92-4301-8465-873eb33f8f28\memmap_d1_400_d2_400_d3_1_order_C_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_8dpf_fed_glucose_625mM_54_20230405\postgavage_vagal_R\mesmerize-batch\f99dc1da-ec92-4301-8465-873eb33f8f28\memmap_d1_400_d2_400_d3_1_order_C_frames_480.mmap


In [16]:
orig = memmap(fish.data_paths['volumes'][str(plane)]['image'])
iw = visualize_images([orig, mcorr_img], names=['original', 'mcorr'])
iw.show()

RFBOutputContext()

In [17]:
iw.plot.canvas.close()

## Optimize parameters with Mesmerize

In [14]:
plane = 1

### Motion correction parameters

In [18]:
strides = [40, 60]  # start a new patch for pw-rigid motion correction every x pixels
overlaps = [15, 25]  # overlap between patches (size of patch strides+overlaps)
max_shifts = [30]  # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = [20, 30]  # maximum shifts deviation allowed for patch with respect to rigid shifts

In [19]:
img_path = fish.data_paths['volumes'][str(plane)]['image']
mes_df = add_mcorr(fish, [img_path], grid=True, 
                         strides=strides, 
                         overlaps=overlaps,
                         max_shifts=max_shifts,
                         max_deviation_rigid=max_deviation_rigid)

Batch does not exist. Creating batch.pickle


In [20]:
mes_df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,8a18cda3-8097-4c8b-915c-2950880ddb47
1,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,81e9b407-0f06-419d-bde6-a34491054277
2,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (25, 25), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,0885bbd4-f303-4098-a4d7-bba35a5187ec
3,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (25, 25), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,5f42942f-05fc-4b3e-a18e-686bb159e2c9
4,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (60, 60), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,e6766df5-1b0f-4231-9c73-6b893b9d2b56
5,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (60, 60), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,0c2e8a37-33a4-4f1a-8542-32074d5c7b4b
6,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (60, 60), 'overlaps': (25, 25), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,db92ebac-64bd-4e2e-a3b5-52b1dea3023e
7,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (60, 60), 'overlaps': (25, 25), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:25:30,None,None,None,5e3a37c4-861d-453d-8b9e-53c0a3352f8f


In [21]:
# See the unique parameters between rows with the same item_name
diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])
diffs

C:\Users\minel\AppData\Local\Temp\ipykernel_30572\688498701.py:2: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])


0    {'overlaps': (15, 15), 'max_deviation_rigid': 20, 'strides': (40, 40)}
1    {'overlaps': (15, 15), 'max_deviation_rigid': 30, 'strides': (40, 40)}
2    {'overlaps': (25, 25), 'max_deviation_rigid': 20, 'strides': (40, 40)}
3    {'overlaps': (25, 25), 'max_deviation_rigid': 30, 'strides': (40, 40)}
4    {'overlaps': (15, 15), 'max_deviation_rigid': 20, 'strides': (60, 60)}
5    {'overlaps': (15, 15), 'max_deviation_rigid': 30, 'strides': (60, 60)}
6    {'overlaps': (25, 25), 'max_deviation_rigid': 20, 'strides': (60, 60)}
7    {'overlaps': (25, 25), 'max_deviation_rigid': 30, 'strides': (60, 60)}
Name: params, dtype: object

### Run motion correction with the given parameter space

In [22]:
mes_df = load_mesmerize(fish)
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

Batch exists. Loading batch.pickle
Running 8a18cda3-8097-4c8b-915c-2950880ddb47 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\8a18cda3-8097-4c8b-915c-2950880ddb47\8a18cda3-8097-4c8b-915c-2950880ddb47-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
finished computing correlation image
Running 81e9b407-0f06-419d-bde6-a34491054277 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\81e9b407-0f06-419d-bde6-a34491054277\81e9b407-0f06-419d-bde6-a34491054277-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
finished computing correlation image
Running 0885bbd4-f303-4098-a4d7-bba35a5187ec with local backend
starting mc
mc finish

In [23]:
# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

True
True
True
True
True
True
True
True


### Visualize outputs

In [26]:
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\8a18cda3-8097-4c8b-915c-2950880ddb47\8a18cda3-8097-4c8b-915c-2950880ddb47-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\81e9b407-0f06-419d-bde6-a34491054277\81e9b407-0f06-419d-bde6-a34491054277-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\0885bbd4-f303-4098-a4d7-bba35a5187ec\0885bbd4-f303-4098-a4d7-bba35a5187ec-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_v

RFBOutputContext()

In [27]:
mcorr_iw.plot.canvas.close()

In [19]:
# Visualize the difference from the mean
mcorr_iw = visualize_diff(mes_df)
mcorr_iw.show()

Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\c9f646b6-e1e0-4d2c-bae1-19cca7c77e23\c9f646b6-e1e0-4d2c-bae1-19cca7c77e23-image_els__d1_166_d2_250_d3_1_order_F_frames_833.mmap
Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\d6f64a99-45e4-4b60-be73-37d7375de6ac\d6f64a99-45e4-4b60-be73-37d7375de6ac-image_els__d1_166_d2_250_d3_1_order_F_frames_833.mmap
Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\e92c5fbe-6669-466a-900e-d9b83e1cf8f6\e92c5fbe-6669-466a-900e-d9b83e1cf8f6-image_els__d1_166_d2_250_d3_1_order_F_frames_833.mmap
Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\7809fe1b-0f60-4cb9-ab8c-cc3084c0e9a3\7809fe1b-0f60-4cb9-ab8c-cc3

RFBOutputContext()

In [20]:
mcorr_iw.plot.canvas.close()

### Choose an index with the best parameters

In [4]:
indices = [1]

In [5]:
df = load_mesmerize(fish)
mes_df = clean_mesmerize(df, indices)
mes_df

Batch exists. Loading batch.pickle


C:\Users\minel\mambaforge\envs\mescore\lib\site-packages\mesmerize_core\caiman_extensions\common.py:209: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
C:\Users\minel\mambaforge\envs\mescore\lib\site-packages\mesmerize_core\caiman_extensions\common.py:209: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
C:\Users\minel\mambaforge\envs\mescore\lib\site-packages\mesmerize_core\caiman_extensions\common

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",{'mean-projection-path': 81e9b407-0f06-419d-bde6-a34491054277\81e9b407-0f06-419d-bde6-a34491054277_mean_projection.n...,2023-05-19T13:25:30,2023-05-19T13:27:10,47.31 sec,None,81e9b407-0f06-419d-bde6-a34491054277


## Run motion correction on all planes

In [6]:
fish.process_mesmerize_filestructure()
mes_df = add_volume(fish, algo='mcorr')
mes_df

Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle


,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",{'mean-projection-path': 81e9b407-0f06-419d-bde6-a34491054277\81e9b407-0f06-419d-bde6-a34491054277_mean_projection.n...,2023-05-19T13:25:30,2023-05-19T13:27:10,47.31 sec,None,81e9b407-0f06-419d-bde6-a34491054277
1,mcorr,img_stack_0,img_stack_0\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,4368db23-af24-4635-b1e5-5ed9917c3b12
2,mcorr,img_stack_1,img_stack_1\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,709f79b0-3e1c-4c70-b33e-67aaa2161c83
3,mcorr,img_stack_2,img_stack_2\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,d75ab8d1-c458-4f99-bf43-6d5aee0e2e92
4,mcorr,img_stack_4,img_stack_4\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,b9273412-6809-414a-8b3e-669f9533dc9c


In [7]:
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

Running 4368db23-af24-4635-b1e5-5ed9917c3b12 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\4368db23-af24-4635-b1e5-5ed9917c3b12\4368db23-af24-4635-b1e5-5ed9917c3b12-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
finished computing correlation image
Running 709f79b0-3e1c-4c70-b33e-67aaa2161c83 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\709f79b0-3e1c-4c70-b33e-67aaa2161c83\709f79b0-3e1c-4c70-b33e-67aaa2161c83-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
finished computing correlation image
Running d75ab8d1-c458-4f99-bf43-6d5aee0e2e92 with local backend
starting mc
mc finished successfully!
computing projecti

### Visualize outputs

In [8]:
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\81e9b407-0f06-419d-bde6-a34491054277\81e9b407-0f06-419d-bde6-a34491054277-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\4368db23-af24-4635-b1e5-5ed9917c3b12\4368db23-af24-4635-b1e5-5ed9917c3b12-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\709f79b0-3e1c-4c70-b33e-67aaa2161c83\709f79b0-3e1c-4c70-b33e-67aaa2161c83-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_v

RFBOutputContext()

In [9]:
mcorr_iw.plot.canvas.close()